<a href="https://colab.research.google.com/github/larissavaladao/time_series_curuai/blob/main/6.%20parameters_by_period/1.landsat_water_period_mosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import pandas as pd
import math
import ee
import geemap
import json

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-curuai2')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import image collections

Landsat 7 - PY6S

In [ ]:
landsat7 = ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD7/Landsat7")\
            .select([ 'B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
print(ee.Date(landsat7.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat7.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

In [ ]:
print(landsat7.aggregate_histogram('WRS_PATH').getInfo())
print(landsat7.aggregate_histogram('WRS_ROW').getInfo())

In [ ]:
print(landsat7.filter(ee.Filter.eq('WRS_PATH',227.0)).aggregate_histogram('WRS_ROW').getInfo())
print(landsat7.filter(ee.Filter.eq('WRS_PATH',228.0)).aggregate_histogram('WRS_ROW').getInfo())

Landsat 8 PY6S

In [ ]:
landsat8 = (ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD8/Landsat8")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
print(ee.Date(landsat8.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat8.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

In [ ]:
print(landsat8.aggregate_histogram('WRS_PATH').getInfo())
print(landsat8.aggregate_histogram('WRS_ROW').getInfo())

In [ ]:
print(landsat8.filter(ee.Filter.eq('WRS_PATH',227.0)).aggregate_histogram('WRS_ROW').getInfo())
print(landsat8.filter(ee.Filter.eq('WRS_PATH',228.0)).aggregate_histogram('WRS_ROW').getInfo())

Landsat 9 PY6S

In [ ]:
landsat9 = (ee.ImageCollection("projects/ee-curuai2/assets/Py6S/LD9/Landsat9")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
print(ee.Date(landsat9.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat9.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

In [ ]:
print(landsat9.aggregate_histogram('WRS_PATH').getInfo())
print(landsat9.aggregate_histogram('WRS_ROW').getInfo())

In [ ]:
print(landsat9.filter(ee.Filter.eq('WRS_PATH',227.0)).aggregate_histogram('WRS_ROW').getInfo())
print(landsat9.filter(ee.Filter.eq('WRS_PATH',228.0)).aggregate_histogram('WRS_ROW').getInfo())

# Transform into remote sensing reflectance and sunglint correction

In [ ]:
def deglint (img):
  '''#dividir imagem corrigida por pi
  Rrs_sat_ac = Rsat_ac / pi
  fazer deglint
  Rrs_sat_ac_deglint(VNIR) = Rrs_sat_ac (VNIR) − Rrs_sat_ac (SWIR)
  correção no artigo INPE CURUAI'''
  Rrs = img.divide(math.pi)
  deglint = Rrs.select(['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2'])\
    .subtract(Rrs.select('swir1'))

  return (deglint
          .copyProperties(img,['system:time_start','CLOUD_COVER',"system:index"]))

## Padronize band names

In [ ]:
name_bands = ['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2']

Landsat 7

In [ ]:
#renomear bandas

ld7 = landsat7.map(lambda img: img.rename(name_bands))
display(ld7.size().getInfo())

Landsat 8

In [ ]:
ld8 = landsat8.map(lambda img: img.rename(name_bands))
display(ld8.size().getInfo())

landsat 9

In [ ]:
ld9 = landsat9.map(lambda img: img.rename(name_bands))
display(ld9.size().getInfo())

In [ ]:
merge_col = ld8.merge(ld7).merge(ld9).sort('system:time_start').map(deglint)
merge_col.limit(5)

In [ ]:
merge_col.size()

# Importar dados de periodo baseado na vazao de obidos

In [ ]:
#import period dates
df_period_limits = pd.read_csv('/content/drive/MyDrive/CURUAI_PROCESS/water_period_limits.csv').drop(columns=['Unnamed: 0'])
# df_period_limits['Data'] = pd.to_datetime(df_period_limits['Data'])
df_period_limits


# Fazer mosaico por periodo - definido por meses

In [ ]:
years = ee.List.sequence(2000,2024);
months = ee.List.sequence(1,12,3);
display(months.getInfo())

In [ ]:
monthly_mosaics =  ee.ImageCollection.fromImages(
    years.map(lambda y: months.map(lambda m: merge_col.filter(ee.Filter.calendarRange(y, y, 'year'))
    .filter(ee.Filter.calendarRange(m, ee.Number(m).add(2), 'month')).median()
    .set('year', y).set('month_init', m).set('month_end', ee.Number(m).add(2)).set('system:time_start', ee.Date.fromYMD(y, m, 1)).set('timestamp', ee.Date.fromYMD(y, m, 1).format('yyyy-MM-dd'))
    )).flatten());
display(monthly_mosaics.size().getInfo())

In [ ]:
monthly_mosaic = monthly_mosaics.map(lambda img: img.set('band_count',img.bandNames().length())).filter(ee.Filter.gt('band_count',0))
display(monthly_mosaic.size().getInfo())
display(monthly_mosaic.limit(2).getInfo())

# Fazer mosaico definido por vazao

In [ ]:
list_images = []
for i in range(0,len(df_period_limits)-1):
  # print(df_period_limits['Data'][i][0:10])
  # print(df_period_limits['Data'][i+1][0:10])

  image = (merge_col.filter(ee.Filter.date(df_period_limits['Data'][i][0:10],df_period_limits['Data'][i+1][0:10]))
              .median()
              .set('year', str(df_period_limits['year'][i]))
              .set('month_init', str(df_period_limits['month'][i]))
              .set('month_end', str(df_period_limits['month'][i+1]))
              .set('system:time_start', ee.Date.parse('yyyy-MM-dd',str(df_period_limits['Data'][i][0:10])))
              .set('time_start', str(df_period_limits['Data'][i][0:10]))
              .set('time_finish', str(df_period_limits['Data'][i+1][0:10]))
  )
  list_images.append(image)



In [ ]:
period_mosaics =  ee.ImageCollection.fromImages(ee.List(list_images))
period_mosaics.limit(2)

In [ ]:
period_mosaics = period_mosaics.map(lambda img: img.set('band_count',img.bandNames().length())).filter(ee.Filter.gt('band_count',0))
display(period_mosaics.size().getInfo())
period_mosaics.limit(2)

# Mask land

In [ ]:
def hsvComposite (image):
    composite = image.select(['blue_mean','green_mean','red_mean']).rgbToHsv()#.clip(limits);
    hue = composite.select("hue");
    max_mask = hue.lte(0.9)
    min_mask = hue.gte(0.3)
    return image.updateMask(max_mask).updateMask(min_mask).select(['blue_mean','green_mean','red_mean','nir_mean']);

# calculate area

In [ ]:
def area_calc(img):
  '''receives an image of curuai and returns water surface area in km2 within the floodplain limits
  as a property of the input image'''

  # Get a pixel area image.
  pixel_area = ee.Image.pixelArea()

  floodplain = ee.FeatureCollection('projects/ee-curuai2/assets/varzea_alagavel')
  image = hsvComposite(img)
  img_mask = image.gt(0)

  areaImage = img_mask.multiply(pixel_area)

  area = areaImage.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': floodplain.geometry(),
    'scale': 30,
    'maxPixels': 1e10
    })
  return image.set('area_km2',ee.Number(area.get('red_mean')).divide(1e6))


In [ ]:
monthly_area = monthly_mosaic.map(area_calc)
monthly_area.first()

In [ ]:
period_area = period_mosaics.map(area_calc)

period_area.limit(3)

# Import Model and Classify Images

In [ ]:
classifier = ee.Classifier.load('projects/ee-curuai2/assets/RF_GEE_regressor')
print(classifier.getInfo())

In [ ]:
predictors = ['blue_mean', 'green_mean','red_mean', 'nir_mean']
spm_classified = monthly_area.select(predictors).map(lambda img: img.addBands(img.classify(classifier=classifier)))
spm_period_classified = period_area.select(predictors).map(lambda img: img.addBands(img.classify(classifier=classifier)))
display(spm_classified.size().getInfo())
display(spm_period_classified.size().getInfo())

In [ ]:
spm_period_classified.limit(5)

In [ ]:
spm_classified.limit(5)

# Export as Asset: Image Collection

In [ ]:
prj = landsat8.first().select('B4').projection().getInfo()
scale = landsat8.first().select('B4').projection().nominalScale().getInfo()

In [ ]:
def export_img(img):
      # define YOUR assetID
    # export

    fname = ee.String(img.get('timestamp')).getInfo()
    export = ee.batch.Export.image.toAsset(\
        image=ee.Image(img),
        description= 'ld_mosaic_'+fname,
        assetId = 'projects/ee-curuai2/assets/landsat_water_period/water_period/mosaic_'+fname,#change the properties to add in images here
        region = ee.FeatureCollection(ee.List([ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry().buffer(30)),ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/points_curuai').geometry().bounds().buffer(30))])).geometry().bounds(),
        crs = prj['crs'],
        scale = scale,
        maxPixels = 1e13)

    # # uncomment to run the export
    export.start()
    print('exporting ' +fname + '--->done')
    return img

In [ ]:
col_length = spm_classified.size().getInfo()
#print(col_length)
# cannot map the function because we run things in the function in bothe client and server sides so we need to make a loop
# for very big time series It is recomendable to break the series and export data in parts
# not only because of the loop but also because of how exporting to assets works in google earth engine
# and you can run into problems if too much information is exported at the same time
for i in range(0,col_length):
    #print(i)
    list = spm_classified.toList(col_length)
    img = ee.Image(list.get(i))
    export_img(img)

# Visualize gif

In [ ]:
# Define arguments for animation function parameters.
video_args = {
    "dimensions": 768,
    "region": ee.FeatureCollection(ee.List([ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry().buffer(30)),ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/points_curuai').geometry().bounds().buffer(30))])).geometry().bounds(),
    "framesPerSecond": 5,
    "bands": ["classification"],
    "min": 1,
    "max": 350,
    "palette": ['blue','green', 'yellow', 'orange','red'],
}

In [ ]:
geemap.download_ee_video(spm_classified, video_args, '/content/classification_SPM.gif')

In [ ]:
geemap.add_text_to_gif(
    '/content/classification_SPM.gif',
    '/content/classification_SPM_text.gif',
    xy=("3%", "5%"),
    text_sequence=spm_classified.aggregate_array('timestamp').getInfo(),
    font_size=30,
    font_color="#ffffff",
    add_progress_bar=True,
)

In [ ]:
# Define arguments for animation function parameters.
video_args = {
    "dimensions": 768,
    "region": ee.FeatureCollection(ee.List([ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry().buffer(30)),ee.Feature(ee.FeatureCollection('projects/ee-curuai2/assets/points_curuai').geometry().bounds().buffer(30))])).geometry().bounds(),
    "framesPerSecond": 5,
    "bands": ["red_mean",'green_mean','blue_mean'],
    "min": 0,
    "max": 0.07,
}

In [ ]:
geemap.download_ee_video(spm_classified, video_args, '/content/water_period.gif')

In [ ]:
geemap.add_text_to_gif(
    '/content/water_period.gif',
    '/content/water_period_text.gif',
    xy=("3%", "5%"),
    text_sequence=spm_classified.aggregate_array('timestamp').getInfo(),
    font_size=30,
    font_color="#ffffff",
    add_progress_bar=True,
)